In [ ]:
import matplotlib.pyplot as plt
import parse_access_om2_results as par
import plot_access_om2_scaling as pas
import plot_pandas_data as ppd
import plot_right_axis_time as pra
from convert_time import days_to_gregorian_years, gregorian_years_to_days

In [ ]:
df = par.parse_om2_dataframe("../access-om2/archive")

In [ ]:
bw = lambda d: d[d["nodesize"] == 28]
cl = lambda d: d[d["nodesize"] == 48]

In [ ]:
df_bw = bw(df)
df_cl = cl(df)

In [ ]:
ocean_cice_ratio_middle = 7

In [ ]:
hi_ice = lambda d: d[d["ocean_ncpus"] / d["ice_ncpus"] <= ocean_cice_ratio_middle]
lo_ice = lambda d: d[d["ocean_ncpus"] / d["ice_ncpus"] > ocean_cice_ratio_middle]

In [ ]:
df_bw_hi = hi_ice(df_bw)
df_bw_lo = lo_ice(df_bw)
df_cl_hi = hi_ice(df_cl)
df_cl_lo = lo_ice(df_cl)

In [ ]:
const_ice_nbr_blocks = 5767

In [ ]:
const_blocks = lambda d: d[d["ice_nbr_blocks"] == const_ice_nbr_blocks]
var_blocks =   lambda d: d[d["ice_nbr_blocks"] != const_ice_nbr_blocks]

In [ ]:
lo_ice_const_blocks = const_blocks(df_cl_lo)
lo_ice_var_blocks = var_blocks(df_cl_lo)

In [ ]:
def plot_ice_block_comparison(plot_fn, x, data,
        xticks=[], 
        yticks=[],
        right_axis_fn=pra.no_right_axis,
        right_yticks=[],
        format_str="{:g}",
        marker='+'):
    if xticks == []:
        xticks = data[x].sort_values().unique()
    xtick_labels = [str(int(val)) for val in xticks]
    fig, ax = plt.subplots()
    plots = [
        plot_fn(
            hi_ice(data),
            ax=ax,
            xticks=xticks,
            yticks=yticks,
            right_axis_fn=right_axis_fn,
            right_yticks=right_yticks,
            marker=marker,
            format_str=format_str)]
    for dataframe in [
        const_blocks(lo_ice(data)),
        var_blocks(lo_ice(data))]:
        plots.append(plot_fn(
            dataframe,
            ax=ax,
            xticks=xticks,
            yticks=yticks,
            marker=marker,
            format_str=format_str))
    legend = (
            "High CICE cores",
            "Low CICE cores, const #blocks",
            "Low CICE cores, var #blocks")
    plt.legend((plot[0] for plot in plots), legend)

In [ ]:
pas.plot_by_xy_labels_fn(
    pas.loglog_om2_speed_by_ncpus,
    "om2_ncpus",
    data=df_bw,
    yticks=[200,500,1000],
    right_yticks=[1,2])

In [ ]:
pas.plot_by_xy_labels_fn(
    pas.loglog_om2_speed_by_ncpus,
    "om2_ncpus",
    data=df_cl_hi,
    yticks=[500,1000,2000],
    right_yticks=[1,2,3,5])

In [ ]:
pas.plot_by_xy_labels_fn(
    pas.semilogx_om2_speed_per_cpu_by_ncpus,
    "om2_ncpus", 
    data=df_cl_hi,
    yticks=[0,0.05,0.1,0.15,0.2],
    right_yticks=[n/10000.0 for n in range(7)])

In [ ]:
pas.plot_by_xy_labels_fn(
    pas.loglog_ocean_speed_by_ncpus,
    "ocean_ncpus", 
    data=df_cl_hi,
    yticks=[500,1000,2000,5000],
    right_yticks=[1,2,5,10],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='d')

In [ ]:
pas.plot_by_xy_labels_fn(
    pas.semilogx_ocean_speed_per_cpu_by_ncpus,    
    "ocean_ncpus",
    data=df_cl_hi,
    right_axis_fn=pra.plot_right_axis_yd,
    yticks=[n/20.0 for n in range(7)],
    right_yticks=[n/5000.0 for n in range(5)],
    marker='d')

In [ ]:
pas.plot_by_xy_labels_fn(
    pas.loglog_ice_speed_by_ncpus, 
    "ice_ncpus", 
    data=df_cl_hi,
    yticks=[500,1000,2000,5000],
    right_yticks=[1,2,5,10,20],
    marker='*')

In [ ]:
pas.plot_by_xy_labels_fn(
    pas.semilogx_ice_speed_per_cpu_by_ncpus,
    "ice_ncpus",
    data=df_cl_hi,
    yticks=[n/5.0 for n in range(8)],
    right_yticks=[n/1000.0 for n in range(8)],
    marker='*')

In [ ]:
pas.plot_by_xy_labels_fn(
    pas.loglog_ice_speed_by_ocean_ncpus_nbr_blocks,
    "ice_nbr_blocks", 
    data=df_cl_hi,
    yticks=[500,1000,2000,3000],
    right_yticks=[1,2,4,8],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='*')

In [ ]:
pas.plot_by_xy_labels_fn(
    pas.loglog_ice_speed_by_ocean_ncpus_nbr_blocks,
    "ice_nbr_blocks", 
    data=df_cl_lo,
    yticks=[500,1000,2000,3000],
    right_yticks=[1,2,4,8],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='*')

In [ ]:
plot_ice_block_comparison(
    pas.loglog_om2_speed_by_ncpus,
    "om2_ncpus",
    df_cl,
    yticks=[500,1000,2000],
    right_yticks=[1, 2, 3, 5],
    right_axis_fn=pra.plot_right_axis_yd)

In [ ]:
plot_ice_block_comparison(
    pas.semilogx_om2_speed_per_cpu_by_ncpus,
    "om2_ncpus",
    df_cl,
    yticks=[0,0.05,0.1,0.15,0.2,0.25],
    right_yticks=[n/10000.0 for n in range(7)],
    right_axis_fn=pra.plot_right_axis_yd)

In [ ]:
plot_ice_block_comparison(
    pas.loglog_ocean_speed_by_ncpus,
    "ocean_ncpus",
    df_cl,
    yticks=[500,1000,2000,5000],
    right_yticks=[1,2,5,10],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='d')

In [ ]:
plot_ice_block_comparison(
    pas.semilogx_ocean_speed_per_cpu_by_ncpus,
    "ocean_ncpus",
    df_cl,
    yticks=[n/20.0 for n in range(7)],
    right_yticks=[n/5000.0 for n in range(5)],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='d')

In [ ]:
plot_ice_block_comparison(
    pas.loglog_ice_speed_by_ncpus,
    "ice_ncpus", 
    df_cl,
    yticks=[500,1000,2000,5000],
    right_axis_fn=pra.plot_right_axis_yd,
    right_yticks=[1,2,5,10],
    marker='*')

In [ ]:
plot_ice_block_comparison(
    pas.semilogx_ice_speed_per_cpu_by_ncpus,
    "ice_ncpus", 
    df_cl,
    yticks=[0,0.5,1,1.5,2],
    right_yticks=[n/1000.0 for n in range(6)],
    right_axis_fn=pra.plot_right_axis_yd,
    marker='*')

In [ ]:
def get_mean(df, x, y):
    mean_y_by_x=dict()
    group_ncpus=df.groupby(x)
    for name, group in group_ncpus:
        means = group.groupby(x)[y].mean()
        mean_y_by_x[name] = means[name]
    return mean_y_by_x

In [ ]:
def round_dict(d, n):
    return {k: round(v, n) for k, v in d.items()}

In [ ]:
def days_to_years_dict(d):
    return {k: days_to_gregorian_years(v) for k, v in d.items()}

In [ ]:
print("ACCESS-OM2 speed per cpu")
print("High")
high_mean = get_mean(df_cl_hi,"om2_ncpus","OM2 speed per cpu")
print("d/d")
print(round_dict(high_mean,2))
print("Y/d")
print(round_dict(days_to_years_dict(high_mean), 5))                        
print("Const")
low_const_mean = get_mean(lo_ice_const_blocks,"om2_ncpus","OM2 speed per cpu")
print("d/d")
print(round_dict(low_const_mean,2))
print("Y/d")
print(round_dict(days_to_years_dict(low_const_mean), 5))                        
print("Var")
low_var_mean = get_mean(lo_ice_var_blocks,"om2_ncpus","OM2 speed per cpu")
print("d/d")
print(round_dict(low_var_mean,2))
print("Y/d")
print(round_dict(days_to_years_dict(low_var_mean), 5))                        

In [ ]:
print("MOM speed per cpu")
print("High")
high_mean = get_mean(df_cl_hi,"ocean_ncpus","Ocean speed per cpu")
print("d/d")
print(round_dict(high_mean,2))
print("Y/d")
print(round_dict(days_to_years_dict(high_mean), 5))                        
print("Const")
low_const_mean = get_mean(lo_ice_const_blocks,"ocean_ncpus","Ocean speed per cpu")
print("d/d")
print(round_dict(low_const_mean,2))
print("Y/d")
print(round_dict(days_to_years_dict(low_const_mean), 5))                        
print("Var")
low_var_mean = get_mean(lo_ice_var_blocks,"ocean_ncpus","Ocean speed per cpu")
print("d/d")
print(round_dict(low_var_mean,2))
print("Y/d")
print(round_dict(days_to_years_dict(low_var_mean), 5)) 

In [ ]:
print("CICE speed per cpu")
print("High")
high_mean = get_mean(df_cl_hi,"ice_ncpus","Ice speed per cpu")
print("d/d")
print(round_dict(high_mean,2))
print("Y/d")
print(round_dict(days_to_years_dict(high_mean), 4))                        
print("Const")
low_const_mean = get_mean(lo_ice_const_blocks,"ice_ncpus","Ice speed per cpu")
print("d/d")
print(round_dict(low_const_mean,2))
print("Y/d")
print(round_dict(days_to_years_dict(low_const_mean), 4))                        
print("Var")
low_var_mean = get_mean(lo_ice_var_blocks,"ice_ncpus","Ice speed per cpu")
print("d/d")
print(round_dict(low_var_mean,2))
print("Y/d")
print(round_dict(days_to_years_dict(low_var_mean), 4)) 

In [ ]:
yticks=[n/10.0 for n in range(5)]
plot_ice_block_comparison(
    pas.semilogx_relative_ice_from_ocn_by_om2_ncpus,
    "om2_ncpus",
    df_cl,
    yticks=yticks,
    marker='*')

In [ ]:
plot_ice_block_comparison(
    pas.semilogx_relative_ocean_oasis_recv_by_om2_ncpus,
    "om2_ncpus",
    df_cl,
    yticks=[n/5.0 for n in range(6)],
    marker='d')

In [ ]:
plot_ice_block_comparison(
    pas.semilogx_relative_wait_by_om2_ncpus,
    "om2_ncpus",
    df_cl,
    yticks=[n/10.0 for n in range(4)])

In [ ]:
plot_ice_block_comparison(
    pas.semilogx_walltime_wait_by_om2_ncpus,
    "om2_ncpus",
    df_cl,
    yticks=[n/10.0 for n in range(6)])

In [ ]:
plot_ice_block_comparison(
    pas.semilogx_om2_init_time_by_om2_ncpus,
    "om2_ncpus", 
    df_cl,
    yticks=[100*n for n in range(6)],
    right_yticks=[n/50.0 for n in range(8)],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='+')

In [ ]:
plot_ice_block_comparison(
    pas.semilogx_ocean_init_time_by_om2_ncpus,
    "om2_ncpus", 
    df_cl,
    yticks=[100*n for n in range(6)],
    right_yticks=[n/50.0 for n in range(9)],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='d')

In [ ]:
plot_ice_block_comparison(
    pas.semilogx_ice_init_time_by_om2_ncpus,
    "om2_ncpus", 
    df_cl,
    yticks=[50*n for n in range(7)],
    right_yticks=[n/50.0 for n in range(5)],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='*')

In [ ]:
max_allowed_init_time = 300.0

In [ ]:
fast_init = lambda d: d[d["Initialization"] <= max_allowed_init_time]
slow_init = lambda d: d[d["Initialization"] > max_allowed_init_time]

In [ ]:
df_cl_fast_init = fast_init(df_cl)
df_cl_slow_init = slow_init(df_cl)

In [ ]:
plot_ice_block_comparison(
    pas.semilogx_om2_init_time_by_om2_ncpus,
    "om2_ncpus", 
    df_cl_fast_init,
    yticks=[50*n for n in range(7)],
    right_yticks=[n/50.0 for n in range(5)],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='+')

In [ ]:
plot_ice_block_comparison(
    pas.semilogx_ocean_init_time_by_om2_ncpus,
    "om2_ncpus", 
    df_cl_fast_init,
    yticks=[20*n for n in range(5)],
    right_yticks=[n/100.0 for n in range(4)],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='d')

In [ ]:
plot_ice_block_comparison(
    pas.semilogx_ice_init_time_by_om2_ncpus,
    "om2_ncpus", 
    df_cl_fast_init,
    yticks=[20*n for n in range(7)],
    right_yticks=[n/100.0 for n in range(4)],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='*')

In [ ]:
plot_ice_block_comparison(
    pas.loglog_ocean_time_per_step_by_om2_ncpus,
    "om2_ncpus",
    df_cl,
    yticks=[0.1,0.2,0.5,1],
    right_yticks=[0.00005,0.0001,0.0002],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='d')

In [ ]:
plot_ice_block_comparison(
    pas.semilogx_ocean_cpu_time_per_step_by_om2_ncpus,
    "om2_ncpus", 
    df_cl,
    yticks=[],
    right_yticks=[n/10.0 for n in range(8)],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='d')

In [ ]:
plot_ice_block_comparison(
    pas.loglog_ice_time_per_step_by_om2_ncpus,
    "om2_ncpus",
    df_cl,
    yticks=[0.1,0.2,0.5,1],
    right_axis_fn=pra.no_right_axis,
    marker='*')

In [ ]:
plot_ice_block_comparison(
    pas.semilogx_ice_cpu_time_per_step_by_om2_ncpus,
    "om2_ncpus", 
    df_cl,
    yticks=[100*n for n in range(7)],
    right_yticks=[n/50.0 for n in range(9)],
    right_axis_fn=pra.plot_right_axis_sh,
    marker='*')